In [1]:
#Eduardo Darrazão
#Rodrigo Faria

In [58]:
import cv2
import numpy as np
import imutils

In [102]:
#
class Shape_Detector:
        def __init__(self):
            pass
        
        def detect(self, c):
            shape = "desconhecido"
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.08*peri, True)
             
            #Se tem quatro vértices
            if len(approx) == 4:
                shape = "quadrilatero"
                
            return shape    

In [107]:
def detecta_palavras(img):
    
    #resized = imutils.resize(img, width=300)
    #ratio = img.shape[0] / float(resized.shape[0])
    
    ret, imgthresh = cv2.threshold(img,170,255,cv2.THRESH_BINARY_INV)
    cv2.imwrite('im1_bin.jpg', imgthresh)
        
    #CV_CHAIN_APPROX_NONE 
    contorno = cv2.findContours(imgthresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    sd = Shape_Detector()    
    
    for c in contorno[0]:
        M = cv2.moments(c)
        shape = sd.detect(c)
        if shape == "quadrilatero" :
            cv2.drawContours(img, [c], -1, (0,255,0), 2)
    
    return img

In [108]:
img1 = cv2.imread("1.jpg", cv2.IMREAD_GRAYSCALE )
cv2.imwrite('im1_gray.jpg', img1)

img1c = detecta_palavras(img1)
cv2.imwrite('im1_detect.jpg', img1c)

True